In [2]:
import re
import string

!pip install -U nltk

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Requirement already up-to-date: nltk in c:\users\cwcol\anaconda3\lib\site-packages (3.4)


You are using pip version 19.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cwcol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cwcol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1) (optional) Scrape 100 Job Listings that contain the title "Data Scientist" from indeed.com

At a minimum your final dataframe of job listings should contain
- Job Title
- Job Description

In [3]:
##### Your Code Here #####
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [4]:
def make_urls(job_title, location,max_jobs):
    num_pages = max_jobs //10
    print(num_pages)
    
    root = 'http://www.indeed.com/jobs?q='
    
    job_title = job_title.replace(" ", "+")
    location = location.replace(' ',"+")
    
    URLs = []
    
    pageNum=0
    for page in range(0,num_pages):
        if pageNum ==0:
            seedURL = root + job_title + '&l=' + location
        else:
            jobNum = pageNum*10
            jobNum = str(jobNum)
            seedURL = root +job_title +'&l=' + location +'&start=' + jobNum
        pageNum = pageNum +1
        URLs.append(seedURL)
    return URLs
    

In [5]:
max_jobs =200
title = '"Data Scientist"'
location = 'United States'

urls = make_urls(title,location, max_jobs)
urls

20


['http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=10',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=20',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=30',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=40',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=50',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=60',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=70',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=80',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=90',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=100',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=110',
 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=120',
 'http://www.indeed.com/jobs?q=

## 2) Use NLTK to tokenize / clean the listings 

In [6]:
df = pd.read_csv("../job_listings.csv")

In [27]:
df.head()

,Unnamed: 0,description,title
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist


In [7]:
l = df["description"][0]
l

'b"<div><div>Job Requirements:</div><ul><li><p>\\nConceptual understanding in Machine Learning models like Nai\\xc2\\xa8ve Bayes, K-Means, SVM, Apriori, Linear/ Logistic Regression, Neural, Random Forests, Decision Trees, K-NN along with hands-on experience in at least 2 of them</p>\\n</li><li><p>Intermediate to expert level coding skills in Python/R. (Ability to write functions, clean and efficient data manipulation are mandatory for this role)</p>\\n</li><li><p>Exposure to packages like NumPy, SciPy, Pandas, Matplotlib etc in Python or GGPlot2, dplyr, tidyR in R</p>\\n</li><li><p>Ability to communicate Model findings to both Technical and Non-Technical stake holders</p>\\n</li><li><p>Hands on experience in SQL/Hive or similar programming language</p>\\n</li><li><p>Must show past work via GitHub, Kaggle or any other published article</p>\\n</li><li><p>Master\'s degree in Statistics/Mathematics/Computer Science or any other quant specific field.</p></li></ul><div><div><div><div><div><d

In [51]:
l2 = re.sub(r'<.*?>'," ", l)

In [58]:
test = "\\ \\\\" 
test =  re.sub(r'\\','!',test)
test

'! !!'

In [78]:
def clean_words(l):
    
    l2 = re.sub(r'<.*?>'," ", l)
    l2 = l2.lower()
    l2 = re.sub(r'\\xc2\\xa8',r'',l2)
    l2 = re.sub(r'\\n',r'',l2)
    l2 = re.sub(r'/',' ', l2)
    l2 = re.sub(r'^b\'','',l2)
    l2 = re.sub(r'^b\"','',l2)
    l2 = re.sub(r"\\'", ' ', l2)
    
    return l2

In [79]:
clean_words(l)

'  job requirements:    conceptual understanding in machine learning models like naive bayes, k-means, svm, apriori, linear  logistic regression, neural, random forests, decision trees, k-nn along with hands-on experience in at least 2 of them    intermediate to expert level coding skills in python r. (ability to write functions, clean and efficient data manipulation are mandatory for this role)    exposure to packages like numpy, scipy, pandas, matplotlib etc in python or ggplot2, dplyr, tidyr in r    ability to communicate model findings to both technical and non-technical stake holders    hands on experience in sql hive or similar programming language    must show past work via github, kaggle or any other published article    master\'s degree in statistics mathematics computer science or any other quant specific field.         apply now         "'

In [80]:
df["clean_words"] = df["description"].apply(lambda x : clean_words(x))

In [81]:
df.head()

,Unnamed: 0,description,title,clean_words,nltk_token
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist,job requirements: conceptual understandin...,"[b, '', job, requirements, :, conceptual, unde..."
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I,"job description as a data scientist 1, you ...","[b, ', job, description, as, a, data, scientis..."
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level,as a data scientist you will be working on c...,"[b, ', as, a, data, scientist, you, will, be, ..."
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist,"$4,969 - $6,756 a month contract under ...","[b, ', $, 4,969, -, $, 6,756, a, month, contra..."
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist,location: usa \xe2\x80\x93 multiple location...,"[b, ', location, :, usa, \xe2\x80\x93, multipl..."


In [99]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

df["nltk_token"] = df["clean_words"].apply(lambda x : word_tokenize(x))

In [100]:
df["nltk_token"] = df["nltk_token"].apply(lambda x: [word for word in x if word.isalpha()])
df["nltk_token"] = df["nltk_token"].apply(lambda x : [word for word in x if not word in stop_words])

In [101]:
df.head()

,Unnamed: 0,description,title,clean_words,nltk_token,word_lemmas
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist,job requirements: conceptual understandin...,"[job, requirements, conceptual, understanding,...","[job, requirement, conceptual, understanding, ..."
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I,"job description as a data scientist 1, you ...","[job, description, data, scientist, help, us, ...","[job, description, a, a, data, scientist, you,..."
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level,as a data scientist you will be working on c...,"[data, scientist, working, consulting, side, b...","[a, a, data, scientist, you, will, be, working..."
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist,"$4,969 - $6,756 a month contract under ...","[month, contract, general, supervision, profes...","[a, month, contract, under, the, general, supe..."
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist,location: usa \xe2\x80\x93 multiple location...,"[location, usa, multiple, locations, years, an...","[location, usa, multiple, location, year, of, ..."


In [102]:
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cwcol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [103]:
lemmatizer = WordNetLemmatizer()

def get_lemmas(word_list):
    lemmas = [lemmatizer.lemmatize(word) for word in word_list]
    
    return lemmas
df["word_lemmas"] = df["nltk_token"].apply(lambda x : get_lemmas(x))

In [104]:
df.head()

,Unnamed: 0,description,title,clean_words,nltk_token,word_lemmas
0,0,"b""<div><div>Job Requirements:</div><ul><li><p>...",Data scientist,job requirements: conceptual understandin...,"[job, requirements, conceptual, understanding,...","[job, requirement, conceptual, understanding, ..."
1,1,b'<div>Job Description<br/>\n<br/>\n<p>As a Da...,Data Scientist I,"job description as a data scientist 1, you ...","[job, description, data, scientist, help, us, ...","[job, description, data, scientist, help, u, b..."
2,2,b'<div><p>As a Data Scientist you will be work...,Data Scientist - Entry Level,as a data scientist you will be working on c...,"[data, scientist, working, consulting, side, b...","[data, scientist, working, consulting, side, b..."
3,3,"b'<div class=""jobsearch-JobMetadataHeader icl-...",Data Scientist,"$4,969 - $6,756 a month contract under ...","[month, contract, general, supervision, profes...","[month, contract, general, supervision, profes..."
4,4,b'<ul><li>Location: USA \xe2\x80\x93 multiple ...,Data Scientist,location: usa \xe2\x80\x93 multiple location...,"[location, usa, multiple, locations, years, an...","[location, usa, multiple, location, year, anal..."


# 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [11]:
##### Your Code Here #####

In [105]:
from nltk.probability import FreqDist

In [106]:
words = [ i for sublist in df["word_lemmas"] for i in sublist ] 

In [107]:
words

['job',
 'requirement',
 'conceptual',
 'understanding',
 'machine',
 'learning',
 'model',
 'like',
 'naive',
 'bayes',
 'svm',
 'apriori',
 'linear',
 'logistic',
 'regression',
 'neural',
 'random',
 'forest',
 'decision',
 'tree',
 'along',
 'experience',
 'least',
 'intermediate',
 'expert',
 'level',
 'coding',
 'skill',
 'python',
 'ability',
 'write',
 'function',
 'clean',
 'efficient',
 'data',
 'manipulation',
 'mandatory',
 'role',
 'exposure',
 'package',
 'like',
 'numpy',
 'scipy',
 'panda',
 'matplotlib',
 'etc',
 'python',
 'dplyr',
 'tidyr',
 'r',
 'ability',
 'communicate',
 'model',
 'finding',
 'technical',
 'stake',
 'holder',
 'hand',
 'experience',
 'sql',
 'hive',
 'similar',
 'programming',
 'language',
 'must',
 'show',
 'past',
 'work',
 'via',
 'github',
 'kaggle',
 'published',
 'article',
 'master',
 'degree',
 'statistic',
 'mathematics',
 'computer',
 'science',
 'quant',
 'specific',
 'field',
 'apply',
 'job',
 'description',
 'data',
 'scientist',
 '

In [108]:
fdist = FreqDist(words)
fdist.most_common(20)


[('data', 4305),
 ('experience', 1979),
 ('team', 1353),
 ('business', 1255),
 ('work', 1209),
 ('science', 978),
 ('learning', 924),
 ('product', 890),
 ('analysis', 816),
 ('model', 801),
 ('analytics', 729),
 ('skill', 722),
 ('machine', 704),
 ('scientist', 704),
 ('opportunity', 582),
 ('solution', 581),
 ('statistical', 576),
 ('ability', 562),
 ('customer', 546),
 ('technology', 536)]

# 4) Visualize the most common word counts

In [109]:
##### Your Code Here #####
from sklearn.feature_extraction.text import CountVectorizer

In [110]:
vectorizer = CountVectorizer(stop_words='english')

In [112]:
vectorizer.fit(words)
print(vectorizer.vocabulary_)

{'job': 3325, 'requirement': 5090, 'conceptual': 1204, 'understanding': 6294, 'machine': 3631, 'learning': 3479, 'model': 3887, 'like': 3533, 'naive': 3977, 'bayes': 576, 'svm': 5931, 'apriori': 363, 'linear': 3541, 'logistic': 3583, 'regression': 4987, 'neural': 4035, 'random': 4864, 'forest': 2478, 'decision': 1560, 'tree': 6202, 'experience': 2238, 'intermediate': 3220, 'expert': 2246, 'level': 3503, 'coding': 1063, 'skill': 5546, 'python': 4808, 'ability': 4, 'write': 6645, 'function': 2557, 'clean': 1017, 'efficient': 1954, 'data': 1519, 'manipulation': 3675, 'mandatory': 3669, 'role': 5221, 'exposure': 2273, 'package': 4297, 'numpy': 4114, 'scipy': 5332, 'panda': 4312, 'matplotlib': 3728, 'dplyr': 1846, 'tidyr': 6091, 'communicate': 1132, 'finding': 2393, 'technical': 6008, 'stake': 5731, 'holder': 2891, 'hand': 2783, 'sql': 5709, 'hive': 2883, 'similar': 5509, 'programming': 4708, 'language': 3432, 'past': 4350, 'work': 6625, 'github': 2667, 'kaggle': 3362, 'published': 4778, 'a

In [123]:
" ".join(df["word_lemmas"][0])

'job requirement conceptual understanding machine learning model like naive bayes svm apriori linear logistic regression neural random forest decision tree along experience least intermediate expert level coding skill python ability write function clean efficient data manipulation mandatory role exposure package like numpy scipy panda matplotlib etc python dplyr tidyr r ability communicate model finding technical stake holder hand experience sql hive similar programming language must show past work via github kaggle published article master degree statistic mathematics computer science quant specific field apply'

In [126]:
df["word_vectors"] = df["word_lemmas"].apply(lambda x : vectorizer.transform([" ".join(x)]))

In [139]:
df["word_vectors"].head()

0      (0, 4)\t2\n  (0, 339)\t1\n  (0, 363)\t1\n  (...
1      (0, 4)\t1\n  (0, 36)\t1\n  (0, 44)\t1\n  (0,...
2      (0, 4)\t1\n  (0, 72)\t1\n  (0, 132)\t1\n  (0...
3      (0, 71)\t1\n  (0, 134)\t1\n  (0, 164)\t1\n  ...
4      (0, 277)\t1\n  (0, 768)\t1\n  (0, 1411)\t1\n...
Name: word_vectors, dtype: object

In [146]:
stack = [ array.toarray() for array in df["word_vectors"]]

In [148]:
stack = np.stack(stack)

In [156]:
stack = stack.reshape(426,6675)

In [154]:
feature_names

['aa',
 'aap',
 'ab',
 'abernathy',
 'ability',
 'able',
 'abounds',
 'abroad',
 'absence',
 'absolutely',
 'absorbing',
 'abstract',
 'abstracting',
 'abstraction',
 'abstractly',
 'abundant',
 'abuse',
 'academia',
 'academic',
 'academy',
 'accelerate',
 'accelerates',
 'accelerating',
 'acceleration',
 'accelerator',
 'accelerometer',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'access',
 'accessibility',
 'accessible',
 'accessing',
 'accidental',
 'accolade',
 'accommodate',
 'accommodating',
 'accommodation',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accordance',
 'according',
 'account',
 'accountability',
 'accountable',
 'accountant',
 'accounting',
 'accredited',
 'accrual',
 'accuracy',
 'accurate',
 'accurately',
 'achieve',
 'achieved',
 'achievement',
 'achieves',
 'achieving',
 'acknowledge',
 'acl',
 'acquire',
 'acquired',
 'acquires',
 'acquiring',
 'acquisition',
 'acre',
 'act',
 'acted',
 'acting',
 'action',
 'actionable',
 'ac

In [157]:
df2 = pd.DataFrame(columns=feature_names,data= stack)

In [159]:
df2.head()

,aa,aap,ab,abernathy,ability,able,abounds,abroad,absence,absolutely,...,zero,zeus,zf,zheng,zillow,zogsports,zone,zoom,zuckerberg,zurich
0,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


 # 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [161]:
##### Your Code Here #####
from sklearn.feature_extraction.text import TfidfVectorizer
text = [x for x in df["word_lemmas"]]
tfidf = TfidfVectorizer(ngram_range=(1,1), max_features=20)
feature_matrix = tfidf.fit_transform(text)
df = pd.DataFrame(feature_matrix.toarray(), columns=feature_names)
print(df.shape)
df.head()

AttributeError: 'list' object has no attribute 'lower'

## Stretch Goals

 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.